In [44]:
#import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
from pyspark.sql import types
from pyspark.sql.functions import col
import requests, pandas as pd, numpy as np
from pandas import DataFrame
from io import StringIO
import time, json
from datetime import date
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# http://stackoverflow.com/questions/28999332/how-to-access-sparkcontext-in-pyspark-script
sc = SparkContext.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Stock_price_prediction').getOrCreate()
df = spark.read.csv('AAPL_stock_Quandl.csv', header = True, inferSchema = True)
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
len(df.columns)

In [ ]:
df.count()

### Feature Engineering

In [ ]:
df = df.withColumn('O-L', df['Open'] - df['Low'])

In [ ]:
from pyspark.sql.functions import lag, lit, lead
from pyspark.sql.window import Window

In [ ]:
df = df.withColumn('dummy_column', lit('Stocks!'))

In [ ]:
w = Window.orderBy("dummy_column")
# Create the lagged value
value_lead = lead('Adj_Close').over(w)
# Add the lagged values to a new coblumn
df = df.withColumn('Next_Adj_Close', value_lead)

In [ ]:
df.show(2)

In [ ]:
value_lag = lag('Adj Close', count=1).over(w)
df = df.withColumn('Avg_close1', value_lag)

value_lag = lag('Adj Close', count=3).over(w)
df = df.withColumn('Adj_Close3', value_lag)

value_lag = lag('Adj Close', count=4).over(w)
df = df.withColumn('Adj_Close4', value_lag)

value_lag = lag('Adj Close', count=4).over(w)
df = df.withColumn('Adj_Close5', value_lag)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import to_date, date_format
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear, dayofweek

In [ ]:
df = df.withColumn('DATEFORMAT', to_date('Date'))

In [ ]:
df = df.withColumn('YEAR', year('DATEFORMAT'))
df = df.withColumn('MONTH', month('DATEFORMAT'))
df = df.withColumn('DAYOFMONTH', dayofmonth('DATEFORMAT'))
df = df.withColumn('DAYOFMONTH', dayofmonth('DATEFORMAT'))
df = df.withColumn('DAYOFWEEK', dayofweek('DATEFORMAT'))
df = df.withColumn('WEEKOFYEAR', weekofyear('DATEFORMAT'))
df = df.withColumn("week", date_format('DATEFORMAT', "W"))

In [ ]:
from pyspark.sql.functions import lag
from pyspark.sql.window import Window

In [ ]:
w = Window().orderBy(df['DATE'])

### Custom Function for finding the moving averages

In [ ]:
from pyspark.sql.functions import coalesce, lit, col, lead, lag
from operator import add
from functools import reduce

def weighted_average(c, window, offsets, weights):
    assert len(weights) == len(offsets)

    def value(i):
        if i < 0: return lag(c, -i).over(window)
        if i > 0: return lead(c, i).over(window)
        return c

    values = [coalesce(value(i) * w, lit(0)) for i, w in zip(offsets, weights)]

    return reduce(add, values, lit(0))

In [ ]:
w = Window.partitionBy("dummy_column").orderBy("DATEFORMAT")
# offsets, delays =  [-1,0], [0.5, 0.5]
a = [1/20] *20
offsets, delays =   [int(e) for e in list(np.arange(20)*-1)] , a
df = df.withColumn("Avg_Close_20", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
a = [1/10] *10
offsets, delays =   [int(e) for e in list(np.arange(10)*-1)] , a
df = df.withColumn("Avg_Close_10", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
a = [1/5] *5
offsets, delays =   [int(e) for e in list(np.arange(5)*-1)] , a
df = df.withColumn("Avg_Close_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
a = [1/80] *80
offsets, delays =   [int(e) for e in list(np.arange(80)*-1)] , a
df = df.withColumn("Avg_Close_80", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1, 8, 15], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_8_15", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1, 3, 5], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_3_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1, 3, 5], [0.25,0.25,0.25,0.25]
df = df.withColumn("Avg_Close_0_1_3_5", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1, 5, 20, 80], [0.2,0.2,0.2,0.2,0.2]
df = df.withColumn("Avg_Close_0_1_5_20_80", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1, 2], [0.333,0.333,0.333]
df = df.withColumn("Avg_Close_3", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
offsets, delays =   [0, 1], [0.5,0.5]
df = df.withColumn("Avg_Close_2", weighted_average(col("DAYOFMONTH"), w, offsets, delays))

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn('O-C', df.Open - df.Adj_Close)
df = df.withColumn('H-L', df.High - df.Low)

In [ ]:
df1 = df.toPandas()
df1 = df1.dropna()
df1['Adj_Close'] = df1['Adj_Close'].astype(float)

In [ ]:
# functions for finding standard deviation, min anx in last n rolling window days
df1['MinClose_5'] = df1['Adj_Close'].rolling(5).min()
df1['MinClose_10'] = df1['Adj_Close'].rolling(10).min()
df1['MinClose_20'] = df1['Adj_Close'].rolling(20).min()
df1['MinClose_80'] = df1['Adj_Close'].rolling(80).min()
df1['MaxClose_5'] = df1['Adj_Close'].rolling(5).max()
df1['MaxClose_10'] = df1['Adj_Close'].rolling(10).max()
df1['MaxClose_20'] = df1['Adj_Close'].rolling(20).max()
df1['MaxClose_80'] = df1['Adj_Close'].rolling(80).max()
df1['StdClose_5'] = df1['Adj_Close'].rolling(5).std()
df1['StdClose_10'] = df1['Adj_Close'].rolling(10).std()
df1['StdClose_20'] = df1['Adj_Close'].rolling(20).std()
df1['StdClose_80'] = df1['Adj_Close'].rolling(80).std()

In [ ]:
df1 = df1.dropna()
df1.head(2)

In [ ]:
cols = df1.columns.drop(['Date','DATEFORMAT', 'dummy_column'],1)
drop_Cols = ['DATEFORMAT', 'dummy_column' ]
df1 = df1.drop(drop_Cols, 1)

In [ ]:
df1[cols] = df1[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
inputCols = list(df1.columns.drop(['Date','Next_Adj_Close']))

In [ ]:
#dropping nans
df1 = df1.dropna()  
df1[pd.isnull(df1).any(axis=1)]

In [ ]:
#created a csv so as to use the processed data with all created set of features
df1.to_csv('final_data.csv')

## machine learning workflow

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df = spark.read.csv('final_data.csv', header = True, inferSchema = True)
# df.printSchema()

In [ ]:
df_assembler = VectorAssembler(inputCols=['High', 'Open', 'Low', 'Adj_Close', 'DAYOFMONTH', 'O-L', 'WEEKOFYEAR', 'Avg_Close_5', 'week', 'DAYOFWEEK', 'Avg_Close_10', 'Avg_Close_20', 'MONTH', 'Volume', 'YEAR'], outputCol="features")
df = df_assembler.transform(df)

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))

## Dividing into Test Train splits 

In [ ]:
test_df = df.where("rank > .8").drop("rank")
train_df = df.where("rank <= .8").drop("rank")

In [ ]:
test_df1 = train_df.toPandas()
test_df1.tail()

In [ ]:
training_df=train_df.select(['features','Next_Adj_Close'])
test_df=test_df.select(['features','Next_Adj_Close'])

In [ ]:
training_df.printSchema()

In [ ]:
training_df.show(1)

### Linear regression Model

In [ ]:
from pyspark.ml.regression import LinearRegression
lin_Reg=LinearRegression(labelCol='Next_Adj_Close')
lr_model = lin_Reg.fit(training_df)
lr_model.intercept
print(lr_model.coefficients)

In [ ]:
training_predictions=lr_model.evaluate(training_df)
training_predictions.meanSquaredError
training_predictions.r2
test_results=lr_model.evaluate(test_df)
#view the residual errors based on predictions 
test_results.residuals.show(10)

In [ ]:
#Test Set results

results=lr_model.evaluate(test_df).predictions

results.select(['Next_Adj_Close','prediction']).show(10,False)

In [ ]:
type(test_results)
test_results.r2
test_results.rootMeanSquaredError
test_results.meanSquaredError
results.count()

## Custom SMAPE function

In [ ]:
# SMAPE Evaluation Metric
from  pyspark.sql.functions import abs
from pyspark.sql import functions as F

results = results.withColumn('Numerator', results.Next_Adj_Close - results.prediction)
results = results.withColumn('Numerator', abs(results.Numerator))
results = results.withColumn('denominator', (results.Next_Adj_Close + results.prediction)/2)
results = results.withColumn('SMAPE', (results.Numerator / results.denominator))
total = results.groupBy().agg(F.sum("SMAPE")).collect()
# total*100/results.count()

In [ ]:
# SMAPE Evaluation Metric
from  pyspark.sql.functions import abs
from pyspark.sql import functions as F

results = results.withColumn('Numerator', abs(results.Next_Adj_Close - results.prediction)/((results.Next_Adj_Close + results.prediction)/2))
total = results.agg(F.sum("SMAPE")).collect()[0][0]
total*100/(results.count())

## Finding Best features using coefficients of Linear Regression

In [ ]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

In [ ]:
best_features_lin = np.array(sorted(range(len(list((lr_model.coefficients)))), key=lambda i: list((lr_model.coefficients))[i], reverse=True)[:15])

In [ ]:
inputCols = pd.Series(inputCols)

In [ ]:
print(list(inputCols[best_features_lin]))

### Random Forest Model - feature selection

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(labelCol='Next_Adj_Close')
rf_model = rf.fit(training_df)
results = rf_model.transform(test_df)

results_rf = rf_model.transform(test_df)
# test_results=rf_model.transform(test_df)

results_rf.select(['Next_Adj_Close','prediction']).show(10,False)

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(results)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
test_results = rf_model.transform(test_df)

## Gradient Boosting Model - feature selection

In [ ]:
from pyspark.ml.regression import GBTRegressor
gbr = GBTRegressor(labelCol='Next_Adj_Close')
gbr_model = gbr.fit(training_df)
results_gbr = gbr_model.transform(test_df)
results_gbr = gbr_model.transform(test_df)

results_gbr.select(['Next_Adj_Close','prediction']).show(10,False)

In [ ]:
results.show()

In [ ]:
evaluator = RegressionEvaluator(
    labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(results_gbr)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
results_gbr.select(['prediction'])

## Logistic Regression Model - classification

In [ ]:
df1 = df.toPandas()
df1 = df1.dropna()
df1['Adj Close'] = df1['Adj Close'].astype(float)
#Created the new Target variable
df1['return'] = np.where(df1['Close'].shift(-1) > df1['Close'],1,0)

In [ ]:
cols = df1.columns.drop(['Next_Adj_Close','Date','DATEFORMAT', 'dummy_column'],1)
drop_Cols = ['DATEFORMAT', 'dummy_column' ]
df[cols] = df1[cols].apply(pd.to_numeric, errors='coerce')
df1 = df1.dropna()  
df1[pd.isnull(df1).any(axis=1)]

In [ ]:
df1.to_csv('final_data_Classification1.csv')
df = spark.read.csv('final_data_Classification1.csv', header = True, inferSchema = True)

In [ ]:
df_assembler = VectorAssembler(inputCols=inputCols, outputCol="features")
df = df_assembler.transform(df)

In [ ]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("Date")))

In [ ]:
# Diving data to test train splits
Data_df = df.where("rank > .60").drop("rank")
Data_df = Data_df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("Date")))

In [ ]:
test_df = Data_df.where("rank > .75").drop("rank")
train_df = Data_df.where("rank <= .75").drop("rank")
test_df.count()

In [ ]:
training_df=train_df.select(['features','return'])
test_df=test_df.select(['features','return'])

In [ ]:
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression(labelCol='return').fit(training_df)
test_results=log_reg.evaluate(test_df).predictions

In [ ]:
train_results=log_reg.evaluate(training_df).predictions

In [ ]:
correct_preds=train_results.filter(train_results['return']==1).filter(train_results['prediction']==1).count()

In [ ]:
#Test Set results
results=log_reg.evaluate(test_df).predictions
results_df = results.select(['return','prediction'])

In [ ]:
correct_preds=results_df.filter(results_df['return']==1).filter(results_df['prediction']==1).count()
#precision on testing dataset 
results_df.filter(results_df['return']==1).count()
float(correct_preds)/(results_df.filter(results_df['prediction']==1).count())

In [ ]:
# df1.info()

In [45]:
df1.to_csv('final_data.csv')

In [46]:
from pyspark.ml.feature import VectorAssembler

## Linear Regression

### taking the data from csv file, this csv file has all the features prepared and we have to repeat the same procedure from this cell to the end by changing the name of csv file form 2weeks to 1month to get results for different time horizons

In [47]:
df = spark.read.csv('/FileStore/tables/final_data_2weeks.csv', header = True, inferSchema = True)
# df.printSchema()

In [48]:
# x = df.toPandas()
# x.tail()

In [49]:
df_assembler = VectorAssembler(inputCols=inputCols, outputCol="features")
df = df_assembler.transform(df)

### converting the date column to DATEFORMAT to vary train sizes just by changing the date

In [50]:
df = df.withColumn('DATEFORMAT', to_date('Date'))

In [51]:
df = df.where((col("DATEFORMAT") < "2018-12-31"))

In [52]:
# x = df.toPandas()
# x.tail() 2018-08-29 

### For a given test data the following code finds the SMAPE and RMSE values for the test data by changing the train dates.

In [53]:
dftest = df.where((col("DATEFORMAT") > "2015-12-31"))
dftrain = df.where((col("DATEFORMAT") < "2015-12-31"))
dftemptemp = dftrain

### looping over through different train dates for calcualting values for different train test splits

In [54]:
#traindates = ["2010-12-31","2008-12-31","2003-12-31","1990-12-31","1980-12-31"]
#traindates = ["2014-12-31","2013-12-31","2010-12-31","2008-12-31","2003-12-31","1990-12-31","1980-12-31"]
#traindates = ["2017-08-31","2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-10-31","2016-10-31","2015-10-31","2013-10-31","2010-10-31","2008-10-31","2003-10-31","1990-10-31","1980-10-31"]
#traindates = ["1980-12-21"]
#traindates = ["2017-06-31","2016-06-31","2015-06-31","2013-06-31","2010-06-31","2008-06-31","2003-06-31","1990-06-31","1980-06-31"]
#traindates = ["2017-07-31","2016-07-31","2015-07-31","2013-07-31","2010-07-31","2008-07-31","2003-07-31","1990-07-31","1980-07-31"]
#traindates = ["2017-09-31","2016-09-31","2015-09-31","2013-09-31","2010-09-31","2008-09-31","2003-09-31","1990-09-31","1980-09-31"]
#traindates = ["2016-11-31","2015-11-31","2013-11-31","2010-11-31","2008-11-31","2003-11-31","1990-11-31","1980-11-31"]
#traindates = ["2017-04-31","2016-04-31","2015-04-31","2013-04-31","2010-04-31","2008-04-31","2003-04-31","1990-04-31","1980-04-31"]
#traindates = ["2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-02-31","2016-02-31","2015-02-31","2013-02-31","2010-02-31","2008-02-31","2003-02-31","1990-02-31","1980-02-31"]
#traindates = ["2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2014-12-15","2013-12-15","2010-12-15","2008-12-15","2003-12-15","1990-12-15","1980-12-15"]
traindates = ["2003-12-31"]

biglist = []
for i in traindates:
  small = []
  dftrai = dftemptemp.where((col("DATEFORMAT") > i))
  training_df=dftrai.select(['features','Next_Adj_Close'])
  test_df=dftest.select(['features','Next_Adj_Close'])
  
  from pyspark.ml.regression import LinearRegression
  lin_Reg=LinearRegression(labelCol='Next_Adj_Close')
  lr_model = lin_Reg.fit(training_df)
  training_predictions=lr_model.evaluate(training_df)
  small.append(training_predictions.rootMeanSquaredError)
  test_results=lr_model.evaluate(test_df)
  small.append(test_results.rootMeanSquaredError)
  
  results=test_results.predictions
  #results = test_results.toPandas()
  small.append(smape(results))
  biglist.append(small)

In [55]:
df1 = training_predictions.predictions.toPandas()
df2 = test_results.predictions.toPandas()
df1 = df2.drop(['features'],axis=1)
df2 = df2.drop(['features'],axis=1)


In [56]:
dates = dftemptemp.toPandas()['Date']

In [57]:
df3 = pd.concat([df1, df2],ignore_index=True)
df3 = 

Out[ 105 ]: 
 Next_Adj_Close prediction
739 146.203695 164.122731
740 156.499589 162.217897
741 155.483940 162.375802
742 155.563599 162.978159
743 157.067158 158.415694

In [58]:
df3.set_index(dates)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2971540086765910> in <module> () 
 ----> 1 df3 . set_index ( dates ) 

 /databricks/python/lib/python3.5/site-packages/pandas/core/frame.py in set_index (self, keys, drop, append, inplace, verify_integrity) 
 2933 index . _cleanup ( ) 
 2934 
 -> 2935 frame . index = index 
 2936 
 2937 if not inplace : 

 /databricks/python/lib/python3.5/site-packages/pandas/core/generic.py in __setattr__ (self, name, value) 
 2755 try : 
 2756 object . __getattribute__ ( self , name ) 
 -> 2757 return object . __setattr__ ( self , name , value ) 
 2758 except AttributeError : 
 2759 pass 

 /databricks/python/lib/python3.5/site-packages/pandas/lib.cpython-35m-x86_64-linux-gnu.so in pandas.lib.AxisProperty.__set__ (pandas/lib.c:44872) () 

 /databricks/python/lib/python3.5/site-packages/pandas/core/generic.py in _set_axis (self, axis, labels) 
 446 
 447 def _set_axis ( self , axis , labels ) : 
 --> 448 self . _data . set_axis ( axis , labels ) 
 449 self . _clear_item_cache ( ) 
 450 

 /databricks/python/lib/python3.5/site-packages/pandas/core/internals.py in set_axis (self, axis, new_labels) 
 2800 raise ValueError('Length mismatch: Expected axis has %d elements, '
 2801 'new values have %d elements' % 
 -> 2802 (old_len, new_len))
 2803 
 2804 self . axes [ axis ] = new_labels 

 ValueError : Length mismatch: Expected axis has 1488 elements, new values have 8760 elements

In [59]:
display(sqlContext.createDataFrame(df3))

Next_Adj_Close,prediction
91.28611427,98.69227182262455
91.40888682,97.03024102109953
90.94612874,95.35602149465694
95.78147847,92.2642029164117
93.91155807,91.68543050973904
94.4309804,92.23057837570036
88.22624451,93.13372602296863
88.85899536,91.66793933585836
91.92830915,93.14312423177512
91.06890129,91.26667348554122


In [60]:
test_results.predictions.show()

+--------------------+--------------+-----------------+
 features|Next_Adj_Close| prediction|
+--------------------+--------------+-----------------+
[96.90531952,99.5...| 91.28611427|98.69227182262455|
[99.87074885,99.9...| 91.40888682|97.03024102109953|
[94.96929082,96.6...| 90.94612874|95.35602149465694|
[93.19381084,94.5...| 95.78147847| 92.2642029164117|
[93.07103829,93.5...| 93.91155807|91.68543050973904|
[93.46768807,93.5...| 94.4309804|92.23057837570036|
[94.95984678,95.0...| 88.22624451|93.13372602296863|
[94.7426338,95.56...| 88.85899536|91.66793933585836|
[92.51383978,94.8...| 91.92830915|93.14312423177512|
[90.85168831,92.2...| 91.06890129|91.26667348554122|
[92.93882169,93.1...| 89.22731301|90.85243350675626|
[89.81284365,92.7...| 90.99334895| 90.7098517765346|
[91.66387596,92.4...| 91.72054022| 89.9518215713565|
[93.14659063,95.8...| 89.27086119|94.45717236454698|
[95.87591889,95.8...| 90.21085431| 92.484393653197|
[94.37431615,95.2...| 90.19186455|92.76144824035339|
[90.70058364,91.2...| 89.50823319|88.04553179680306|
[88.57567409,89.2...| 88.96702504| 88.1712214375209|
[89.52007833,91.9...| 89.24237655|91.25770853320387|
[91.10667746,91.3...| 91.75851974|89.94222666001838|
+--------------------+--------------+-----------------+
only showing top 20 rows

In [61]:
#smape(results)

### sMAPE function for pandas dataframes

In [62]:
def smape(df):
  df['Numerator'] = df.Next_Adj_Close - df.prediction
  df['Numerator'] = df['Numerator'].abs()
  df['Denominator'] = ((df.prediction + df.Next_Adj_Close)/2)
  df['total'] = df['Numerator'] /df['Denominator']
  total = df['total'].sum()*100/(df['total'].count())
  return total 

## sMAPE function for spark pandas dataframe, both functions work fine

In [63]:
# SMAPE Evaluation Metric
from  pyspark.sql.functions import abs
from pyspark.sql import functions as F
def smape(results):
  results = results.withColumn('Numerator', results.Next_Adj_Close - results.prediction)
  results = results.withColumn('Numerator', abs(results.Numerator))
  results = results.withColumn('denominator', (results.Next_Adj_Close + results.prediction)/2)
  results = results.withColumn('SMAPE', (results.Numerator / results.denominator))
  total = results.groupBy().agg(F.sum("SMAPE")).collect()
  x = total[0][0]*100/(results.count())
  
  return x

In [64]:
twoyears = pd.DataFrame(biglist, index=traindates, columns=['trian RMSE','test RMSE','SMAPE'])

In [65]:
twoyears

Out[ 75 ]: 
 trian RMSE test RMSE SMAPE
2014-12-15 4.218275 20.642610 10.977871
2013-12-15 4.165917 10.975168 5.359071
2010-12-15 3.839994 9.766168 4.601233
2008-12-15 3.430462 8.989794 4.103345
2003-12-15 2.828810 8.588474 3.809661
1990-12-15 2.032883 8.542866 3.702072
1980-12-15 1.738899 8.547802 3.687520

In [66]:
smape(results)

0.8741012960193892
 Out[ 182 ]: 2.2412853744086902

## Cross validation and hyperparameter tuning for Linear regression

### user defined function for rolling window cross validation with 10 folds

In [67]:
def tuning_lr(df1,df2,parameters = [0.0,1.0]):  
  training_df=df1.select(['features','Next_Adj_Close'])
  test_df=df2.select(['features','Next_Adj_Close'])  
  smapevalues = []  
  for i in parameters:
    lin_Reg=lr(labelCol='Next_Adj_Close', regParam = i)
    lr_model = lin_Reg.fit(training_df )
    test_results=lr_model.evaluate(test_df)
    results=test_results.predictions
    smapevalues.append(smape(results))
    #smapevalues.append(test_results.rootMeanSquaredError)
  return np.array(smapevalues)


In [68]:
def crossval(df,hyperparameters):  
  df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))
  temp1 = df.where((col("rank") > 0.0) & (col("rank") < 0.1)).drop("rank")
  temp2 = df.where((col("rank") > 0.1) & (col("rank") < 0.2)).drop("rank")
  temp3 = df.where((col("rank") > 0.2) & (col("rank") < 0.3)).drop("rank")
  temp4 = df.where((col("rank") > 0.3) & (col("rank") < 0.4)).drop("rank")
  temp5 = df.where((col("rank") > 0.4) & (col("rank") < 0.5)).drop("rank")
  temp6 = df.where((col("rank") > 0.5) & (col("rank") < 0.6)).drop("rank")
  temp7 = df.where((col("rank") > 0.6) & (col("rank") < 0.7)).drop("rank")
  temp8 = df.where((col("rank") > 0.7) & (col("rank") < 0.8)).drop("rank")
  temp9 = df.where((col("rank") > 0.8) & (col("rank") < 0.9)).drop("rank")
  temp10 = df.where((col("rank") > 0.9) & (col("rank") < 1.0)).drop("rank")  
  temp = [temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,temp10]  
  Allresults = []
  tempresults = [0] * len(parameters)
  tempresults = np.array(tempresults)  
  i = 0
  j = 3
  while j < 10:
    tempresults = np.add(tempresults,tuning_lr(temp[i].unionAll(temp[i+1]).unionAll(temp[i+2]),temp[j],parameters))
    i = i +1
    j = j+1  
  #return parameters[np.argmax(tempresults)]
  return tempresults
  

In [69]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark.ml.regression import LinearRegression as lr
# dftest = df.where((col("DATEFORMAT") > "2018-06-21"))
dftrain = df.where((col("DATEFORMAT") < "2018-06-21"))
#dftemptemp = dftrain

In [70]:
#dftrain.toPandas().head()

### giving a list of hyperparameter values for linear regression where C values range from o to 50000

In [71]:
parameters = [0.0, 0.0001, 0.0005,0.001,0.005, 0.01,0.05, 0.1, 0.5, 0.75,1.0,2.0,5.0,10.0,50.0,100.0,500.0,5000.0,50000.0]
smapeallvalues = crossval(dftrain,parameters)

In [72]:
smapeallvalues*(1/7)

Out[ 152 ]: 
array([ 4.68618889, 4.68826844, 4.68354096, 4.68262267,
 4.69561109, 4.71992032, 4.94938693, 5.20379718,
 6.59738038, 7.31475256, 7.98517143, 10.34181405,
 15.49882036, 21.16097297, 39.11508481, 48.32239586,
 69.58270638, 86.15637749, 89.13693504])

The best C value is 0.0001

In [73]:
parameters[np.argmin(smapeallvalues)]

Out[ 166 ]: 0.001

### Using best paremerter to see its performance on test data. The tuning improved the performance by decreasing the sMAPE value from 1.055 to 1.046

In [74]:
dftest = df.where((col("DATEFORMAT") > "2015-12-31"))
dftrain = df.where((col("DATEFORMAT") < "2015-12-31"))  
lin_Reg=lr(labelCol='Next_Adj_Close', regParam = 0.0001)
lr_model = lin_Reg.fit(dftrain)
test_results=lr_model.evaluate(dftest)
results=test_results.predictions
print(smape(results))

1.046187977468177

In [75]:
dftest = df.where((col("DATEFORMAT") > "2018-08-31"))
dftrain = df.where((col("DATEFORMAT") < "2018-08-31"))  
lin_Reg=lr(labelCol='Next_Adj_Close', regParam = 0.0005)
lr_model = lin_Reg.fit(dftrain)
test_results=lr_model.evaluate(dftest)
results=test_results.predictions
print(smape(results))

## Random Forests 

In [77]:
df = spark.read.csv('/FileStore/tables/final_data_2weeks.csv', header = True, inferSchema = True)
# df.printSchema()

In [78]:
df_assembler = VectorAssembler(inputCols=inputCols, outputCol="features")
df = df_assembler.transform(df)

In [79]:
df = df.withColumn('DATEFORMAT', to_date('Date'))

In [80]:
df = df.where((col("DATEFORMAT") < "2018-12-31"))

In [81]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from  pyspark.sql.functions import abs
from pyspark.sql import functions as F

In [82]:
dftest = df.where((col("DATEFORMAT") > "2018-08-15"))
dftrain = df.where((col("DATEFORMAT") < "2018-08-15"))
dftemptemp = dftrain

In [83]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from  pyspark.sql.functions import abs
from pyspark.sql import functions as F


#traindates = ["2014-12-31","2013-12-31","2010-12-31","2008-12-31","2003-12-31","1990-12-31","1980-12-31"]
#traindates = ["2017-08-31","2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-10-31","2016-10-31","2015-10-31","2013-10-31","2010-10-31","2008-10-31","2003-10-31","1990-10-31","1980-10-31"]
#traindates = ["1980-12-21"]
#traindates = ["2017-06-31","2016-06-31","2015-06-31","2013-06-31","2010-06-31","2008-06-31","2003-06-31","1990-06-31","1980-06-31"]
#traindates = ["2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2016-11-31","2015-11-31","2013-11-31","2010-11-31","2008-11-31","2003-11-31","1990-11-31","1980-11-31"]
#traindates = ["2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-04-31","2016-04-31","2015-04-31","2013-04-31","2010-04-31","2008-04-31","2003-04-31","1990-04-31","1980-04-31"]
#traindates = ["2017-02-31","2016-02-31","2015-02-31","2013-02-31","2010-02-31","2008-02-31","2003-02-31","1990-02-31","1980-02-31"]
traindates = ["2017-08-15","2016-08-15","2015-08-15","2013-08-15","2010-08-15","2008-08-15","2003-08-15","1990-08-15","1980-08-15"]

biglist = []
for i in traindates:
  small = []
  dftrai = dftemptemp.where((col("DATEFORMAT") > i))
  training_df=dftrai.select(['features','Next_Adj_Close'])
  test_df=dftest.select(['features','Next_Adj_Close'])
  
  rf = RandomForestRegressor(labelCol='Next_Adj_Close')
  rf_model = rf.fit(training_df)

  training_predictions= rf_model.transform(training_df)
  evaluator = RegressionEvaluator(
  labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
  rmse = evaluator.evaluate(training_predictions)
  small.append(rmse)
  
  test_results=rf_model.transform(test_df)
  evaluator = RegressionEvaluator(
  labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
  rmse = evaluator.evaluate(test_results)
  small.append(rmse)
  
  #results=test_results.prediction
  results = test_results.toPandas()
  small.append(smape(results))
  biglist.append(small)

In [84]:
fourmonths = pd.DataFrame(biglist, index=traindates, columns=['trian RMSE','test RMSE','SMAPE'])

In [85]:
fourmonths

## using user defined function to perform cross validation and tuning for random forests

In [86]:
def tuning_lr(df1,df2,parameters):
  
  training_df=df1.select(['features','Next_Adj_Close'])
  test_df=df2.select(['features','Next_Adj_Close'])

  trees = parameters[0]
  depth = parameters[1]
  
  smapevalues = []
  
  for i in trees:
    print(i)
    for j in depth:
      rf = RandomForestRegressor(numTrees=i, maxDepth=j, labelCol='Next_Adj_Close')
      rf_model = rf.fit(training_df)  
      test_results=rf_model.transform(test_df)
      results = test_results.toPandas()
      smapevalues.append(smape(results))

    #smapevalues.append(test_results.rootMeanSquaredError)
    
    
  return np.array(smapevalues)

In [87]:
def crossval(df,parameters):
  
  df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))
    
  temp1 = df.where((col("rank") > 0.0) & (col("rank") < 0.1)).drop("rank")
  temp2 = df.where((col("rank") > 0.1) & (col("rank") < 0.2)).drop("rank")
  temp3 = df.where((col("rank") > 0.2) & (col("rank") < 0.3)).drop("rank")
  temp4 = df.where((col("rank") > 0.3) & (col("rank") < 0.4)).drop("rank")
  temp5 = df.where((col("rank") > 0.4) & (col("rank") < 0.5)).drop("rank")
  temp6 = df.where((col("rank") > 0.5) & (col("rank") < 0.6)).drop("rank")
  temp7 = df.where((col("rank") > 0.6) & (col("rank") < 0.7)).drop("rank")
  temp8 = df.where((col("rank") > 0.7) & (col("rank") < 0.8)).drop("rank")
  temp9 = df.where((col("rank") > 0.8) & (col("rank") < 0.9)).drop("rank")
  temp10 = df.where((col("rank") > 0.9) & (col("rank") < 1.0)).drop("rank")
  
  temp = [temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,temp10]
  
  Allresults = []
  #tempresults = [0] * len(parameters)
  tempresults = [0] * len(parameters[0])*len(parameters[1])
  tempresults = np.array(tempresults)
  
  i = 0
  j = 3
  while j < 10:
    tempresults = np.add(tempresults,tuning_lr(temp[i].unionAll(temp[i+1]).unionAll(temp[i+2]),temp[j],parameters))
    i = i +1
    j = j+1
  
  #return parameters[np.argmax(tempresults)]
  return tempresults
  

In [88]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
from pyspark.ml.regression import LinearRegression as lr

dftemptrain = df.where((col("DATEFORMAT") > "2013-08-31"))
dftraintuning = dftemptrain.where((col("DATEFORMAT") < "2018-08-31"))
dftest = df.where((col("DATEFORMAT") > "2018-08-31"))

### hyperparameterse considered are n_estimators and max_depth

In [89]:
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# max_depth = [int(x) for x in np.linspace(1, 10, num = 5)]
n_estimators = [10,100]
max_depth = [3,10]

In [90]:
parameters = [n_estimators,max_depth]
trees = parameters[0]
depth = parameters[1]  
listedparameters = []
for i in trees:
  for j in depth:
    listedparameters.append([i,j])
#listedparameters
# max_depth
parameters = [n_estimators,max_depth]
# parameters[1]
#default numTrees=20,maxDepth=5

In [91]:
smapeallvalues = crossval(dftraintuning,parameters)

In [92]:
smapeallvalues*(1/7)

In [93]:
np.min(smapeallvalues)*(1/7)

### finding the best hyper parameter

In [94]:
listedparameters[np.argmin(smapeallvalues)]

### using the best paramter obtained to check its performance on test data

In [95]:
training_df=dftraintuning.select(['features','Next_Adj_Close'])
test_df=dftest.select(['features','Next_Adj_Close'])

rf = RandomForestRegressor(numTrees=100, maxDepth=10,labelCol='Next_Adj_Close')
rf_model = rf.fit(training_df)

training_predictions= rf_model.transform(training_df)
evaluator = RegressionEvaluator(
labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
print(evaluator.evaluate(training_predictions))

test_results=rf_model.transform(test_df)
evaluator = RegressionEvaluator(
labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
print(evaluator.evaluate(test_results))

#results=test_results.prediction
results = test_results.toPandas()
print(smape(results))

## Gradient Boosting

In [96]:
df = spark.read.csv('/FileStore/tables/final_data_2weeks.csv', header = True, inferSchema = True)
# df.printSchema()

In [97]:
df_assembler = VectorAssembler(inputCols=inputCols, outputCol="features")
df = df_assembler.transform(df)

In [98]:
df = df.withColumn('DATEFORMAT', to_date('Date'))

In [99]:
df = df.where((col("DATEFORMAT") < "2018-12-31"))

In [100]:
dftest = df.where((col("DATEFORMAT") > "2018-08-15"))
dftrain = df.where((col("DATEFORMAT") < "2018-08-15"))
dftemptemp = dftrain

In [101]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

#traindates = ["2014-12-31","2013-12-31","2010-12-31","2008-12-31","2003-12-31","1990-12-31","1980-12-31"]
#traindates = ["2017-08-31","2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-10-31","2016-10-31","2015-10-31","2013-10-31","2010-10-31","2008-10-31","2003-10-31","1990-10-31","1980-10-31"]
#traindates = ["1980-12-21"]
#traindates = ["2017-06-31","2016-06-31","2015-06-31","2013-06-31","2010-06-31","2008-06-31","2003-06-31","1990-06-31","1980-06-31"]
#traindates = ["2016-11-31","2015-11-31","2013-11-31","2010-11-31","2008-11-31","2003-11-31","1990-11-31","1980-11-31"]
#traindates = ["2016-08-31","2015-08-31","2013-08-31","2010-08-31","2008-08-31","2003-08-31","1990-08-31","1980-08-31"]
#traindates = ["2017-08-21","2016-08-21","2015--08-21","2013--08-21","2010--08-21","2008--08-21","2003--08-21","1990--08-21","1980--08-21"]
#traindates = ["2017-04-31","2016-04-31","2015-04-31","2013-04-31","2010-04-31","2008-04-31","2003-04-31","1990-04-31","1980-04-31"]
#traindates = ["2017-02-31","2016-02-31","2015-02-31","2013-02-31","2010-02-31","2008-02-31","2003-02-31","1990-02-31","1980-02-31"]
traindates = ["2017-08-15","2016-08-15","2015-08-15","2013-08-15","2010-08-15","2008-08-15","2003-08-15","1990-08-15","1980-08-15"]

biglist = []
for i in traindates:
  small = []
  dftrai = dftemptemp.where((col("DATEFORMAT") > i))
  training_df=dftrai.select(['features','Next_Adj_Close'])
  test_df=dftest.select(['features','Next_Adj_Close'])
  
  gbr = GBTRegressor(labelCol='Next_Adj_Close')
  gbr_model = gbr.fit(training_df)

  training_predictions= gbr_model.transform(training_df)
  evaluator = RegressionEvaluator(
  labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
  rmse = evaluator.evaluate(training_predictions)
  small.append(rmse)
  
  test_results=gbr_model.transform(test_df)
  evaluator = RegressionEvaluator(
  labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
  rmse = evaluator.evaluate(test_results)
  small.append(rmse)
  
  #results=test_results.prediction
  results = test_results.toPandas()
  small.append(smape(results))
  biglist.append(small)

In [102]:
fourmonths = pd.DataFrame(biglist, index=traindates, columns=['trian RMSE','test RMSE','SMAPE'])

In [103]:
fourmonths

### User defined function for hyperparameter tuning and cross validation for gradient boosting

In [104]:
def tuning_lr(df1,df2,parameters):
  
  training_df=df1.select(['features','Next_Adj_Close'])
  test_df=df2.select(['features','Next_Adj_Close'])

  trees = parameters[0]
  depth = parameters[1]
  
  smapevalues = []
  
  for i in trees:
    for j in depth:
      gbr = GBTRegressor(maxIter=i, maxDepth=j, labelCol='Next_Adj_Close')
      gbr_model = gbr.fit(training_df)  
      test_results=gbr_model.transform(test_df)
      results = test_results.toPandas()
      smapevalues.append(smape(results))

    #smapevalues.append(test_results.rootMeanSquaredError) 
    
  return np.array(smapevalues)

In [105]:
def crossval(df,parameters):
  
  df = df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))
    
  temp1 = df.where((col("rank") > 0.0) & (col("rank") < 0.1)).drop("rank")
  temp2 = df.where((col("rank") > 0.1) & (col("rank") < 0.2)).drop("rank")
  temp3 = df.where((col("rank") > 0.2) & (col("rank") < 0.3)).drop("rank")
  temp4 = df.where((col("rank") > 0.3) & (col("rank") < 0.4)).drop("rank")
  temp5 = df.where((col("rank") > 0.4) & (col("rank") < 0.5)).drop("rank")
  temp6 = df.where((col("rank") > 0.5) & (col("rank") < 0.6)).drop("rank")
  temp7 = df.where((col("rank") > 0.6) & (col("rank") < 0.7)).drop("rank")
  temp8 = df.where((col("rank") > 0.7) & (col("rank") < 0.8)).drop("rank")
  temp9 = df.where((col("rank") > 0.8) & (col("rank") < 0.9)).drop("rank")
  temp10 = df.where((col("rank") > 0.9) & (col("rank") < 1.0)).drop("rank")
  
  temp = [temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,temp10]
  
  Allresults = []
  #tempresults = [0] * len(parameters)
  tempresults = [0] * len(parameters[0])*len(parameters[1])
  tempresults = np.array(tempresults)
  
  i = 0
  j = 3
  while j < 10:
    tempresults = np.add(tempresults,tuning_lr(temp[i].unionAll(temp[i+1]).unionAll(temp[i+2]),temp[j],parameters))
    i = i +1
    j = j+1
  
  #return parameters[np.argmax(tempresults)]
  return tempresults
  

In [106]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

dftemptrain = df.where((col("DATEFORMAT") > "2013-08-31"))
dftraintuning = dftemptrain.where((col("DATEFORMAT") < "2018-08-31"))

### hyparameters considered n_estimators and max_depth

In [107]:
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# max_depth = [int(x) for x in np.linspace(1, 10, num = 5)]
n_estimators = [10,20,50,100,200,500]
max_depth = [3,5,10,15]

In [108]:
parameters = [n_estimators,max_depth]
trees = parameters[0]
depth = parameters[1]  
listedparameters = []
for i in trees:
  for j in depth:
    listedparameters.append([i,j])
#listedparameters
# max_depth
parameters = [n_estimators,max_depth]
# parameters[1]
# default maxDepth=5, maxIter=20

In [109]:
smapeallvalues = crossval(dftraintuning,parameters)
smapeallvalues*(1/7)

### finding the best parameter for gradient boost trees

In [110]:
parameters[np.argmin(smapeallvalues)]

### checking the performance of best parameter on test data

In [111]:
from pyspark.ml.regression import GBTRegressor
training_df=dftraintuning.select(['features','Next_Adj_Close'])
test_df=dftest.select(['features','Next_Adj_Close'])

gbr = GBTRegressor(maxIter=20, maxDepth=5, labelCol='Next_Adj_Close')
gbr_model = gbr.fit(training_df)

training_predictions= gbr_model.transform(training_df)
evaluator = RegressionEvaluator(
labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
print(evaluator.evaluate(training_predictions))

test_results=gbr_model.transform(test_df)
evaluator = RegressionEvaluator(
labelCol="Next_Adj_Close", predictionCol="prediction", metricName="rmse")
print(evaluator.evaluate(test_results))

#results=test_results.prediction
results = test_results.toPandas()
print(smape(results))